In [5]:
!pip install gradio

In [6]:
import gradio as gr
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

labels = ["pozytywny", "neutralny", "negatywny"]

def predict_sentiment(text):  
   result = classifier(text, candidate_labels=labels)
   label = result['labels'][0]
   score = result['scores'][0]
   return f"Sentyment: **{label}** (pewność: {score:.2%})"




Device set to use cpu


In [7]:

interface = gr.Interface(
  fn=predict_sentiment,
  inputs=gr.Textbox(lines=2, placeholder="Wpisz zdanie finansowe..."),
  outputs="text",
  title="Klasyfikator Sentimentu Finansowego (BART)",
  description="Ten interfejs wykorzystuje model BART w trybie zero-shot do klasyfikacji sentymentu w zdaniach finansowych. Etykiety: pozytywny, neutralny, negatywny."
)

interface.launch(server_name="0.0.0.0")


* Running on local URL:  http://0.0.0.0:7861
* To create a public link, set `share=True` in `launch()`.


In [ ]:
from openai import OpenAI

openai_api_key = "sk-"  # ← wstaw swój klucz
client = OpenAI(api_key=openai_api_key)

completion = client.chat.completions.create(
 model="gpt-3.5-turbo",
 messages=[
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": "Tell me a Joke!"}
 ]
)
print(completion.choices[0].message)



ChatCompletionMessage(content="Sure! Why don't scientists trust atoms?\n\nBecause they make up everything!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [11]:
def openai_zero_shot_classify(text, candidate_labels):
   prompt = f"Klasyfikuj następujący tekst do podanych kategorii: {candidate_labels}.\nTekst: \"{text}\"\n\nKategoria:"

   completion = client.chat.completions.create(
       model="gpt-3.5-turbo",
       messages=[ {"role": "system", "content": "Jesteś asystentem, który klasyfikuje tekst do podanych kategorii. Odpowiadaj tylko nazwą kategorii."}, {"role": "user", "content": prompt}],
       temperature=0
   )
   predicted_label = completion.choices[0].message.content.strip()
   return f"Kategoria: **{predicted_label}**"
labels = "mieszkanie sprzedaż,mieszkanie wynajem,dom sprzedaż,dom wynajem"


In [18]:
import pandas as pd
import kagglehub

rag_worksop = kagglehub.dataset_download('martininf1n1ty/rag-workshop')

df = pd.read_csv(f'{rag_worksop}/adresowo_offers_all_categories.csv').head(10)
df.head()


,category,offer_url,price,currency,num_images,street_address,description
0,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-wola...,NaN,NaN,4,"ul. Księcia Janusza, Warszawa Wola, mazowieckie",Posiadam na sprzedaż przestronne mieszkanie na...
1,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-targ...,NaN,NaN,14,"ul. Toruńska, Warszawa Targówek, mazowieckie","Na sprzedaż duże, jasne mieszkanie o powierzch..."
2,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-bemo...,NaN,NaN,20,"ul. Aleksandra Świętochowskiego, Warszawa Bemo...",Szukasz klimatycznego mieszkania w otoczeniu z...
3,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ocho...,NaN,NaN,6,"ul. Bitwy Warszawskiej 1920 r., Warszawa Ochot...",Mieszkanie po generalnym remoncie. Do zamieszk...
4,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ursu...,NaN,NaN,14,"ul. Tadeusza Hennela, Warszawa Ursus, mazowieckie",Zaoszczędź 16 000 zł i kup mieszkanie bez pośr...


In [17]:
openai_zero_shot_classify(df['description'][0], labels)

'Kategoria: **mieszkanie sprzedaż**'

In [19]:
df['category_pl']  = df['description'].apply(lambda x: openai_zero_shot_classify(x, labels))


In [21]:
df

,category,offer_url,price,currency,num_images,street_address,description,category_pl
0,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-wola...,NaN,NaN,4,"ul. Księcia Janusza, Warszawa Wola, mazowieckie",Posiadam na sprzedaż przestronne mieszkanie na...,Kategoria: **mieszkanie sprzedaż**
1,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-targ...,NaN,NaN,14,"ul. Toruńska, Warszawa Targówek, mazowieckie","Na sprzedaż duże, jasne mieszkanie o powierzch...",Kategoria: **mieszkanie sprzedaż**
2,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-bemo...,NaN,NaN,20,"ul. Aleksandra Świętochowskiego, Warszawa Bemo...",Szukasz klimatycznego mieszkania w otoczeniu z...,Kategoria: **mieszkanie sprzedaż**
3,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ocho...,NaN,NaN,6,"ul. Bitwy Warszawskiej 1920 r., Warszawa Ochot...",Mieszkanie po generalnym remoncie. Do zamieszk...,Kategoria: **mieszkanie sprzedaż**
4,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ursu...,NaN,NaN,14,"ul. Tadeusza Hennela, Warszawa Ursus, mazowieckie",Zaoszczędź 16 000 zł i kup mieszkanie bez pośr...,Kategoria: **mieszkanie sprzedaż**
5,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-wloc...,NaN,NaN,13,"ul. Obywatelska, Warszawa Włochy, mazowieckie","Sprzedaż tylko bezpośrednia, pośrednikom dzięk...",Kategoria: **mieszkanie sprzedaż**
6,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-wloc...,NaN,NaN,7,"ul. Trzcinowa, Warszawa Włochy, mazowieckie",NaN,Kategoria: **Brak danych**
7,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-bial...,NaN,NaN,10,"ul. Daniszewska, Warszawa Białołęka, mazowieckie",Na sprzedaż: 3-pokojowe mieszkanie z dużym bal...,Kategoria: **mieszkanie sprzedaż**
8,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ursy...,NaN,NaN,8,"ul. Indiry Gandhi, Warszawa Ursynów, mazowieckie",Sprzedam bezpośrednio kawalerkę o powierzchni ...,Kategoria: **mieszkanie sprzedaż**
9,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-srod...,NaN,NaN,6,"ul. Nowogrodzka, Warszawa Śródmieście, mazowie...",Do sprzedania mieszkanie w ścisłym centrum prz...,Kategoria: **mieszkanie sprzedaż**


In [ ]:
from openai import OpenAI

token = "sk-"
# klient OpenAI
client = OpenAI(api_key=token)

# otwarcie pliku audio
with open("ElevenLabs_2025-11-14T15_22_25_Jerzy_pvc_sp100_s50_sb75_se28_m2.mp3", "rb") as audio_file:
   transcription = client.audio.transcriptions.create(
       model="whisper-1",  # albo "whisper-1"
       file=audio_file
   )

print("=== Transkrypcja ===")
print(transcription.text)


=== Transkrypcja ===
Litwo, ojczyzno moja, Ty jesteś jak zdrowie, Ile Cię trzeba cenić, ten tylko się dowie, kto Cię stracił. Dziś piękność Twą w całej ozdobie widzę i opisuję, Bo tęsknię po Tobie, Panno Święta, co jasnej bronisz Częstochowy I w ostrej świecisz bramie. Ty, co gród zamkowy nowogródzki ochraniasz z jego wiernym ludem, Jak mnie, dziecko, do zdrowia powróciłaś cudem, Gdy od płaczącej matki pod Twoją opiekę, Ofiarowany martwą podniosłem powiekę I zaraz mogłem pieszo do Twych świątyń progu Iść za wrócone życie podziękować Bogu. Tak nas powrócisz cudem na ojczyzny łono, Tymczasem przenoś moją duszę utęsknioną Do tych pagórków leśnych, do tych łąk zielonych, Szeroko nad błękitnym niemnem rozciągnionych, Do tych pól malowanych zbożem rozmaitem, Wyzłacanych pszenicą posrebrzanych żytem, Gdzie bursztynowy świeżop gryka jak śnieg biała, Gdzie anieńskim rumieńcem dzięcielina pała, A wszystko przepasane jakby wstęgą miedzą zieloną, Na niej z rzadka ciche grusze siedzą.


In [25]:
from transformers import pipeline

# pipeline do transkrypcji audio (Whisper)
pipe = pipeline(
   task="automatic-speech-recognition",
   model="openai/whisper-small",   # możesz zmienić na medium/large
)

# transkrypcja z pliku
result = pipe("ElevenLabs_2025-11-14T15_22_25_Jerzy_pvc_sp100_s50_sb75_se28_m2.mp3", return_timestamps="word")

print("=== Transkrypcja (Whisper) ===")
print(result["text"])




Device set to use cpu
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


=== Transkrypcja (Whisper) ===
 Litwo, ojczyzno moja, ty jesteś jak zdrowie, ile cię trzeba cenić, ten tylko się dowie, kto cię stracił. Dziś piękność twą w całej ozdobie widzę i opisuję, bo tęsknie po tobie Panno Święta, co jasnej bronisz, częstochowy i w ostrej świecisz bramie. Ty, co grud zamkowy, nowogrócki ochraniasz z jego wiernym ludem, jak mnie dziecko do zdrowia powróciłaś cudem, Gdy odpłaczącej matki pod twoją opiekę, ofiarowany, martwą podniosłem powiekę i zaraz mogłem pieszo do twych świątyń progu iść zawrócone życie podziękować Bogu. Tak nas powrócisz, cudem na ojczyzny łono, tymczasem przenoś moją duszem utęsknioną do tych pagórków leśnych, do tych łąk zielonych, szeroko nad błękitnym niemnym rozciągnionych, do tych pól malowanych z bożem rozmaitem, wyzłacanych przenicom posrebrzanych rzytem, gdzie bursztynowy świeżop gryka jak śnieg biała, gdzie anieńskim rumieńcem dziencielina pała, a wszystko przepasane jakby wstęgą Miedzą zieloną na niej z rzadka ciche grusze siedzą.


# Zadanie 1 – Klasyfikacja obrazów z CLIP

Uruchom kod ładujący model CLIP (Contrastive Language–Image Pretraining).
Model pozwala na klasyfikację obrazów bez trenowania — wystarczy podać opisy (prompty) kandydatów w języku naturalnym.

#### Instrukcja

* Przygotuj model CLIP do predykcji co jest na zdjęciu.

* Wczytaj podany obraz scena.jpg z użyciem polecenia wget — możesz też użyć własnego pliku lub poniższego znaku drogowego.

* Zdefiniuj własny zestaw etykiet tekstowych, np.:

```python
labels = [
   "a photo of a stop sign",
   "a photo of a speed limit 50 sign",
   "a photo of a no entry sign and a pedestrian crossing sign",
   "a photo of a pedestrian crossing sign",
   "a photo of a yield sign",
   "a photo of a priority road sign"
]
```

Sprawdź, jak zmienia się predykcja modelu po zmianie promptów (np. dodaniu szczegółów:
"a photo of a red stop sign" zamiast "a photo of a traffic sign").

In [27]:
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [gdown]


In [28]:
#link do danych: https://drive.google.com/file/d/1v5t_d03F2RKovxPoChTguKxuGwnXpgR7/view?usp=sharing

import gdown

file_id = '1v5t_d03F2RKovxPoChTguKxuGwnXpgR7'
output_filename = 'scena.jpg'

gdown.download(id=file_id, output=output_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1v5t_d03F2RKovxPoChTguKxuGwnXpgR7
To: /home/student/HybridRag/2_Praca_z_API,_retry_i_streaming.pptx_-_Prezentacje_Google/wykład/scena.jpg
100%|██████████| 136k/136k [00:00<00:00, 1.03MB/s]


'scena.jpg'

In [29]:
import torch
from transformers import pipeline

clip = pipeline(
   task="zero-shot-image-classification",
   model="openai/clip-vit-base-patch32",
   dtype=torch.bfloat16,
   device=0  # jeśli masz GPU; w przeciwnym razie użyj device=-1
)



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [ ]:
labels = [
   "a photo of a stop sign",
   "a photo of a speed limit 50 sign",
   "a photo of a no entry sign",
   "a photo of a pedestrian crossing sign",
   "a photo of a yield sign",
   "a photo of a priority road sign"
]
results = clip("scena.jpg", candidate_labels=labels)

In [ ]:
clip("scena.jpg", candidate_labels=labels)

[{'score': 0.83984375, 'label': 'a photo of a pedestrian crossing sign'},
 {'score': 0.078125, 'label': 'a photo of a priority road sign'},
 {'score': 0.0537109375, 'label': 'a photo of a yield sign'},
 {'score': 0.0252685546875, 'label': 'a photo of a no entry sign'},
 {'score': 0.0034332275390625, 'label': 'a photo of a speed limit 50 sign'},
 {'score': 0.00098419189453125, 'label': 'a photo of a stop sign'}]

In [33]:
labels = [
    'modern finish',
    'traditional finish',
    'in need of renovation',
    'luxury finish',
    'basic finish'
]
clip("image_00001.jpg", candidate_labels=labels)

[{'score': 0.76953125, 'label': 'modern finish'},
 {'score': 0.0810546875, 'label': 'traditional finish'},
 {'score': 0.0810546875, 'label': 'luxury finish'},
 {'score': 0.06298828125, 'label': 'basic finish'},
 {'score': 0.00457763671875, 'label': 'in need of renovation'}]

In [34]:
labels = [
    'modern finish',
    'traditional finish',
    'in need of renovation',
    'luxury finish',
    'basic finish'
]
clip("image_00002.jpg", candidate_labels=labels)

[{'score': 0.6171875, 'label': 'modern finish'},
 {'score': 0.1767578125, 'label': 'basic finish'},
 {'score': 0.08349609375, 'label': 'in need of renovation'},
 {'score': 0.07373046875, 'label': 'traditional finish'},
 {'score': 0.050537109375, 'label': 'luxury finish'}]


### Zadanie 2 – Transkrypcja audio z OpenAI

* Otwórz plik audio Alloy_tts-1_1x_2024-10-28T09_42_03-535Z.mp3.

* Wykonaj transkrypcję przy pomocy API OpenAI (gpt-4o-mini-transcribe lub whisper-1).



In [3]:
#link do danych: https://drive.google.com/file/d/1_518XiGq9hF9-6umkHIkAEGlXMpmQqvc/view?usp=sharing

import gdown

file_id = '1_518XiGq9hF9-6umkHIkAEGlXMpmQqvc'
output_filename = 'Alloy_tts-1_1x_2024-10-28T09_42_03-535Z.mp3'

gdown.download(id=file_id, output=output_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1_518XiGq9hF9-6umkHIkAEGlXMpmQqvc
To: /content/Alloy_tts-1_1x_2024-10-28T09_42_03-535Z.mp3
100%|██████████| 495k/495k [00:00<00:00, 91.9MB/s]


'Alloy_tts-1_1x_2024-10-28T09_42_03-535Z.mp3'

In [ ]:
!


### Zadanie 3 – Transkrypcja audio z modelem HuggingFace (Whisper)

* Uruchom pipeline automatic-speech-recognition z modelem openai/whisper-small.

Porównaj wynik transkrypcji z wynikiem API OpenAI z Zadania 3.

Odpowiedz na pytania:

* Który model lepiej rozpoznał tekst?

* Czy występują różnice w interpunkcji i zapisie?



### Zadanie 4. Użycie PaddleOCR do analizy obrazu w języku polskim

Korzystając z biblioteki `PaddleOCR`:

1. Wczytaj obrazek z polskim tekstem (np. plik `Screenshot 2025-11-12 at 15.25.56.png`).
2. Uruchom OCR z parametrem `lang='pl'`.
3. Wypisz rozpoznany tekst w konsoli.

In [35]:
#link do danych: https://drive.google.com/file/d/1npxluHj4P5NVmFes5opL8k16-sWGm2rH/view?usp=sharing

import gdown

file_id = '1npxluHj4P5NVmFes5opL8k16-sWGm2rH'
output_filename = 'Screenshot 2025-11-12 at 15.25.56.png'

gdown.download(id=file_id, output=output_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1npxluHj4P5NVmFes5opL8k16-sWGm2rH
To: /home/student/HybridRag/2_Praca_z_API,_retry_i_streaming.pptx_-_Prezentacje_Google/wykład/Screenshot 2025-11-12 at 15.25.56.png
100%|██████████| 1.73M/1.73M [00:00<00:00, 6.97MB/s]


'Screenshot 2025-11-12 at 15.25.56.png'

In [36]:
!pip install paddlepaddle paddleocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 101.5 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.5/759.5 kB 40.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 98.4 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 67.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 57.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.1/782.1 kB 44.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.8 MB/s  0:00:00
  Attempting uninstall: PyYAML90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/24 [shapely]
    Found existing installation: PyYAML 6.0.3━━━━━━━━━━━━━━━━━  4/24 [shapely]
    Uninstalling PyYAML-6.0.3:━━━━━━━━━━━━━━━━━━━━━━━

In [38]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(
   use_doc_orientation_classify=False,
   use_doc_unwarping=False,
   use_textline_orientation=False,
   lang='pl')

# Run OCR inference on a sample image
result = ocr.predict(
   input="Screenshot 2025-11-12 at 15.25.56.png")

extracted_text = ""
if result and result[0] and 'rec_texts' in result[0]:
   extracted_text = " ".join(result[0]['rec_texts'])

print(extracted_text)




Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/student/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/student/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


STARBUCKS ZMIEÑ DZIEN NALEPSZE CARA MACC Starbucks Cheled Lont a. Czekamy na Ciebie wled LVBET BET Google Maps Yiaaa


In [ ]:
import pandas as pd
import kagglehub

rag_worksop = kagglehub.dataset_download('martininf1n1ty/rag-workshop')

df = pd.read_csv(f'{rag_worksop}/adresowo_offers_all_categories.csv')
df.head()


Using Colab cache for faster access to the 'rag-workshop' dataset.


,category,offer_url,price,currency,num_images,street_address,description
0,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-wola...,NaN,NaN,4,"ul. Księcia Janusza, Warszawa Wola, mazowieckie",Posiadam na sprzedaż przestronne mieszkanie na...
1,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-targ...,NaN,NaN,14,"ul. Toruńska, Warszawa Targówek, mazowieckie","Na sprzedaż duże, jasne mieszkanie o powierzch..."
2,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-bemo...,NaN,NaN,20,"ul. Aleksandra Świętochowskiego, Warszawa Bemo...",Szukasz klimatycznego mieszkania w otoczeniu z...
3,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ocho...,NaN,NaN,6,"ul. Bitwy Warszawskiej 1920 r., Warszawa Ochot...",Mieszkanie po generalnym remoncie. Do zamieszk...
4,mieszkania_sprzedaz,https://adresowo.pl/o/mieszkanie-warszawa-ursu...,NaN,NaN,14,"ul. Tadeusza Hennela, Warszawa Ursus, mazowieckie",Zaoszczędź 16 000 zł i kup mieszkanie bez pośr...
